**Setup code**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K

# import numpy to create arrays for trial runs
import numpy as np

# matplotlib for visualisations
import matplotlib.pyplot as plt

# if using Colab, for later saving of models and loading data
from google.colab import drive



Mount Google Drive

In [ ]:
#Mount Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Pre-Processing**


In [ ]:
BasePath = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray'

TrainPath = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray/train'
TestPath = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray/test'

TrainPathNormal = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray/train/NORMAL'
TrainPathPneu = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray/train/PNEUMONIA'

TestPathNormal = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray/test/NORMAL'
TestPathPneu = '/content/gdrive/MyDrive/BMET5933/Assignment2/chest_xray/test/PNEUMONIA'

#2 ways to do it
#- split pne into bacteria and viral
#- you have 3 classes and have to compare against normal


#- or group all togheter and comapre normal against pneuo

In [ ]:
import tensorflow as tf
import numpy as np
import os
import PIL
import glob
import pathlib
import os
from os import listdir
from os.path import isfile, join
import imghdr

def image_dataset_from_dict(data_dict, image_size=(224, 224), batch_size=32, shuffle=True):
    filenames = list(data_dict.keys())
    labels = list(data_dict.values())
    num_samples = len(filenames)

    # Create a dataset from the filenames and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    # Map function to load and preprocess the images
    def load_and_preprocess_image(filename, label):
        image = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, image_size)
        image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
        return image, label

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=num_samples)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

# Your code to create the dictionaries
TrainBacterial = [filename for filename in os.listdir(TrainPathPneu) if filename.startswith("BACTERIA-")]
TestBacterial = [filename for filename in os.listdir(TestPathPneu) if filename.startswith("BACTERIA-")]
TrainViral = [filename for filename in os.listdir(TrainPathPneu) if filename.startswith("VIRUS-")]
TestViral = [filename for filename in os.listdir(TestPathPneu) if filename.startswith("VIRUS-")]
TrainNormal = [f for f in listdir(TrainPathNormal) if isfile(join(TrainPathNormal, f))]
TestNormal =  [f for f in listdir(TestPathNormal) if isfile(join(TestPathNormal, f))]

Train_labels = {}
Test_labels = {}

for image in TrainBacterial:
    if imghdr.what(os.path.join(TrainPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Train_labels[os.path.join(TrainPathPneu, image)] = 0

for image in TrainViral:
    if imghdr.what(os.path.join(TrainPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Train_labels[os.path.join(TrainPathPneu, image)] = 1

for image in TrainNormal:
    if imghdr.what(os.path.join(TrainPathNormal, image)) in ["jpeg", "png", "gif", "bmp"]:
        Train_labels[os.path.join(TrainPathNormal, image)] = 2

for image in TestBacterial:
    if imghdr.what(os.path.join(TestPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Test_labels[os.path.join(TestPathPneu, image)] = 0

for image in TestViral:
    if imghdr.what(os.path.join(TestPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Test_labels[os.path.join(TestPathPneu, image)] = 1

for image in TestNormal:
    if imghdr.what(os.path.join(TestPathNormal, image)) in ["jpeg", "png", "gif", "bmp"]:
        Test_labels[os.path.join(TestPathNormal, image)] = 2

# Create the dataset using the function and dictionaries
train_dataset = image_dataset_from_dict(Train_labels, image_size=(224, 224), batch_size=32, shuffle=True)
validation_dataset = image_dataset_from_dict(Test_labels, image_size=(224, 224), batch_size=32, shuffle=True)

print(train_dataset)

In [ ]:
print(len(train_dataset))
print(len(validation_dataset))

**Model Deeper Model**

In [ ]:
import tensorflow as tf
import numpy as np
import os
import PIL
import glob
import pathlib
import os
from os import listdir
from os.path import isfile, join
import imghdr
from scipy.ndimage import median_filter

def image_dataset_from_dict(data_dict, image_size=(224, 224), batch_size=32, shuffle=True):
    filenames = list(data_dict.keys())
    labels = list(data_dict.values())
    num_samples = len(filenames)

    # Create a dataset from the filenames and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    # Map function to load and preprocess the images
    def load_and_preprocess_image(filename, label):
        image = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32) / 255.0  # Convert to float32 and normalize to [0, 1]

        # Apply histogram equalization
        image = histogram_equalization(image)

        # Apply median filtering
        image = median_filtering(image)

        image.set_shape((None, None, 3))  # Set the shape explicitly

        image = tf.image.resize(image, image_size)
        return image, label

    def histogram_equalization(image):
        # Convert image to HSV color space
        image_hsv = tf.image.rgb_to_hsv(image)

        # Extract the V channel
        image_v = image_hsv[..., 2]

        # Expand dimensions to match the expected rank
        image_v = tf.expand_dims(image_v, axis=-1)

        # Apply histogram equalization
        image_v = tf.image.adjust_contrast(image_v, contrast_factor=2.0)

        # Remove the extra dimensions
        image_v = tf.squeeze(image_v, axis=-1)

        # Combine the modified V channel with the original H and S channels
        image_hsv_equalized = tf.stack([image_hsv[..., 0], image_hsv[..., 1], image_v], axis=-1)

        # Convert the image back to RGB color space
        image_equalized = tf.image.hsv_to_rgb(image_hsv_equalized)

        return image_equalized

    def median_filtering(image):
        # Apply median filtering with a kernel size of 3x3
        image_filtered = tf.numpy_function(median_filter, [image, (3, 3)], tf.float32)
        return image_filtered

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=num_samples)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset


# Your code to create the dictionaries
TrainBacterial = [filename for filename in os.listdir(TrainPathPneu) if filename.startswith("BACTERIA-")]
TestBacterial = [filename for filename in os.listdir(TestPathPneu) if filename.startswith("BACTERIA-")]
TrainViral = [filename for filename in os.listdir(TrainPathPneu) if filename.startswith("VIRUS-")]
TestViral = [filename for filename in os.listdir(TestPathPneu) if filename.startswith("VIRUS-")]
TrainNormal = [f for f in listdir(TrainPathNormal) if isfile(join(TrainPathNormal, f))]
TestNormal =  [f for f in listdir(TestPathNormal) if isfile(join(TestPathNormal, f))]

Train_labels = {}
Test_labels = {}

for image in TrainBacterial:
    if imghdr.what(os.path.join(TrainPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Train_labels[os.path.join(TrainPathPneu, image)] = 0

for image in TrainViral:
    if imghdr.what(os.path.join(TrainPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Train_labels[os.path.join(TrainPathPneu, image)] = 1

for image in TrainNormal:
    if imghdr.what(os.path.join(TrainPathNormal, image)) in ["jpeg", "png", "gif", "bmp"]:
        Train_labels[os.path.join(TrainPathNormal, image)] = 2

for image in TestBacterial:
    if imghdr.what(os.path.join(TestPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Test_labels[os.path.join(TestPathPneu, image)] = 0

for image in TestViral:
    if imghdr.what(os.path.join(TestPathPneu, image)) in ["jpeg", "png", "gif", "bmp"]:
        Test_labels[os.path.join(TestPathPneu, image)] = 1

for image in TestNormal:
    if imghdr.what(os.path.join(TestPathNormal, image)) in ["jpeg", "png", "gif", "bmp"]:
        Test_labels[os.path.join(TestPathNormal, image)] = 2

# Create the dataset using the function and dictionaries
train_dataset = image_dataset_from_dict(Train_labels, image_size=(224, 224), batch_size=32, shuffle=True)
validation_dataset = image_dataset_from_dict(Test_labels, image_size=(224, 224), batch_size=32, shuffle=True)

print(train_dataset)

In [ ]:
import tensorflow as tf
from tensorflow import keras

def create_model(input_shape, num_classes):
    # Define the base model
    base_model = keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling="avg"
    )

    # Freeze the base model's weights
    base_model.trainable = False

    # Data augmentation
    data_augmentation = keras.Sequential([
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.2),
        keras.layers.experimental.preprocessing.RandomZoom(0.2),
    ])

    # Add a classification head
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = keras.layers.Dense(128, activation="relu")(x)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    return model

# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 3

# Create the model
model = create_model(input_shape, num_classes)

# Train the model using the dataset
model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

In [ ]:
import tensorflow as tf
from tensorflow import keras

def create_model(input_shape, num_classes):
    # Define the base model
    base_model = keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling="avg"
    )

    # Set the last several layers to be trainable
    num_layers_to_freeze = 150
    for layer in base_model.layers[:-num_layers_to_freeze]:
        layer.trainable = False

    # Data augmentation
    data_augmentation = keras.Sequential([
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.2),
        keras.layers.experimental.preprocessing.RandomZoom(0.2),
    ])

    # Add a classification head
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = keras.layers.Dense(128, activation="relu")(x)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    return model

# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 3

# Create the model
model = create_model(input_shape, num_classes)

# Train the model using the dataset with fine-tuning
model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

In [ ]:
def make_model_test_deeper(input_shape, num_classes):
  # specify the size of the input
  inputs = keras.Input(shape=input_shape)

  # recale from 0-255 to 0-1
  x = layers.experimental.preprocessing.Rescaling(1.0/255) (inputs)
  x = layers.Conv2D(32, (3,3), strides=1, padding="same") (x)
  x = layers.Activation("relu") (x)
  x = layers.Conv2D(32, (3,3), strides=1, padding="same") (x)
  x = layers.Activation("relu") (x)
  x = layers.MaxPooling2D(pool_size=3) (x)
  x = layers.Conv2D(64, (3,3), strides=1, padding="same") (x)
  x = layers.Activation("relu") (x)
  x = layers.Conv2D(64, (3,3), strides=1, padding="same") (x)
  x = layers.Activation("relu") (x)
  x = layers.MaxPooling2D(pool_size=3) (x)
  x = layers.Flatten() (x)

  # OFTEN NEED A FLATTEN AS THE INPUT TO DENSE
  # output is a Dense with softmax
  outputs = layers.Dense(num_classes, activation="softmax") (x)

  # return the model
  return keras.Model(inputs, outputs)


NUM_CLASSES=3
image_size = (224, 224, 3)
model_test_deeper = make_model_test_deeper(input_shape=image_size, num_classes=NUM_CLASSES)


# set some hyperparameters
LEARNING_RATE = 0.0001
BATCH_SIZE = 32 # should be the same as the dataset batch size
NUM_EPOCHS = 10  # how many epochs to train for (each epoch visits the training data once)

# compile the model
model_test_deeper.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

train_history = model_test_deeper.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=validation_dataset
)

In [ ]:
acc = train_history.history['accuracy']
val_acc = train_history.history['val_accuracy']

loss = train_history.history['loss']
val_loss = train_history.history['val_loss']

epochs_range = range(NUM_EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(validation_dataset)
print('Test accuracy:', test_acc)

https://link-springer-com.ezproxy.library.sydney.edu.au/article/10.1007/s00521-023-08450-y

This procedure requires the application of the MakeSense data labelling tool. MakeSense is a free online application for image labelling. It draws polygons to completely outline images’ objects. For this study, labels are saved as “.json” files for mask R-CNN, and the “.vgg” format for faster R-CNN. In this study, a total of 4000 images, including 3200 training images and 800 validation images, are labelled.